Install qiskit
and also matplotlib and pylatexenc (for nicer circuit plots)

In [ ]:
!pip install qiskit
!pip install qiskit_aerqiskit_ibm_provider
!pip install qiskit_ibm_runtime
!pip install qiskit_ibmq_provider
!pip install matplotlib
!pip install pylatexenc 

ERROR: Could not find a version that satisfies the requirement qiskit_aerqiskit_ibm_provider (from versions: none)
ERROR: No matching distribution found for qiskit_aerqiskit_ibm_provider


  Obtaining dependency information for qiskit_ibm_runtime from https://files.pythonhosted.org/packages/da/65/53d962154a36f571f552138379dd7ebddac34275606b321feda5f253e6df/qiskit_ibm_runtime-0.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for requests-ntlm>=1.1.0 from https://files.pythonhosted.org/packages/9e/5d/836b97537a390cf811b0488490c389c5a614f0a93acb23f347bd37a2d914/requests_ntlm-1.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client>=1.5.1 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb7537a3/websocket_client-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for ibm-platform-services>=0.22.6 from https://files.pythonhosted.org/packages/8e/03/988c48bbb5a7dd554f14e252822b5342111ce51fb25b56f09aff33ace46c/ibm_platform_services-0.57.2-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=2.5.0 from https://files.pythonhosted.org/packages/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.76 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.3 which is incompatible.


### Import Qiskit

In [2]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, transpile
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator
from qiskit.visualization import plot_histogram


ImportError: cannot import name 'execute' from 'qiskit' (C:\Users\padlewsk\AppData\Roaming\Python\Python39\site-packages\qiskit\__init__.py)

## Problem 1: A first quantum circuit in Qiskit

Initialise a simple circuit with 1 qubit and 1 classical bit for the output

In [7]:
# Every quantum circuit is initialised with every qubit in |0> 
qc1a = QuantumCircuit(QuantumRegister(1), ClassicalRegister(1))
# qc1a = QuantumCircuit(1,1)

In [ ]:
# In Qiskit, standard gates are methods of the circuit object


# TODO apply a single qubit gate to qubit 0

qc1a.x(0)
#qc1a.h(0)
#qc1a.s(0)
#qc1a.t(0)


qc1a.measure(qubit=0, cbit=0)

we can also draw the circuit

In [ ]:
qc1a.draw('mpl')

Finially we run the circuit on a simulator (1000 times)
and plot the measurement results

In [ ]:
simulator = QasmSimulator()
results1a = execute(qc1a, backend=simulator, shots=1000).result()

In [ ]:
plot_histogram(results1a.get_counts())

#### Preparing Bell states

In [ ]:
# Shortcut for QuantumCircuit(QuantumRegister(2), ClassicalRegister(2))
qc1b = QuantumCircuit(2,2)


# TODO prepare a Bell state
# e.g. by doing a h gate on qubit 0 followed by a cnot on qubits 0 and 1

qc1b.h(0)
qc1b.cnot(0,1)


qc1b.measure([0,1], [0,1])
# Shortcut:
#qc1b.measure_all()

qc1b.draw('mpl')

In [ ]:
results1b = execute(qc1b, backend=simulator, shots=1000000).result()
plot_histogram(results1b.get_counts())

## Problem 2: Use different simulators in Qiskit

We are going to see how the use of different simulators affects the final result

In [9]:
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator
from qiskit.providers.aer.noise import NoiseModel

To use the Noise Model from real quantum devices (or run circuits on real hardware)
you can create an account on https://quantum-computing.ibm.com

On the Welcome page you can find the API Token.

In [10]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', token=<IBM Quantum API key>)

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
)

Get the noise model for a real quantum device

First print the available ones

In [ ]:
service.backends(simulator=False)

Choose one and get a noise model (approximately) describing it

In [ ]:
backend = service.get_backend('ibm_perth')
back_prop = backend.properties()
noise_model = NoiseModel.from_backend_properties(back_prop)
print(noise_model)

prepare the simulators

In [13]:
statevector_simulator = StatevectorSimulator()
qasm_simulator        = QasmSimulator()
noisy_qasm_simulator  = QasmSimulator(noise_model=noise_model)

In [ ]:
qc2 = QuantumCircuit(4)


# TODO implement a circuit that prepares 1/√2 (|0000⟩ + |1111⟩)

qc2.h(0)
qc2.cnot(0,1)
qc2.cnot(0,2)
qc2.cnot(0,3)


qc2.draw('mpl')

If we use the statevector_simulator we can directly extract the coefficients in the computational basis:

In [ ]:
# Statevector simulator is the exact state at the end of the circuit, then shots=1 by default
results2 = execute(qc2, backend=statevector_simulator).result()
results2.get_statevector()

As we can see, we obtain the vector describing the state of the quantum computer.

Now we add the missing measurements of the end of the circuit

In [ ]:
qc2.measure_all()
qc2.draw('mpl')

In [17]:
results2 = execute(qc2, backend=qasm_simulator, shots=1000).result()

plot the result of 1000 runs

In [ ]:
plot_histogram(results2.get_counts())

and finally we run it using the simulated noise model from the device we selected

In [ ]:
results2 = execute(qc2, backend=noisy_qasm_simulator, shots=1000).result()
plot_histogram(results2.get_counts())

## Problem 3: Transpile a quantum Circuit

In [20]:
from qiskit.compiler import transpile

First we select a backend (in this case a real device) to transpile for

In [21]:
from qiskit.providers.fake_provider import FakePerth

fake_hardware_backend = FakePerth()
hardware_backend = service.get_backend("ibm_perth")

You can look at the different devices in the IBM Quantum portal

In [ ]:
qc3 = QuantumCircuit(5)


# TODO implement a cirquit of your choice using 5 qubits

thetas = [0.1, 0.2, 0.3, 0.4, 0.5]

for i,theta  in enumerate(thetas):
    qc3.ry(theta, i)
qc3.cnot(0,1)
qc3.cnot(0,2)
qc3.cnot(0,3)
qc3.cnot(0,4)

qc3.barrier() # to put a barrier in the circuit for visualisation purposes
for i in range(5):
    qc3.h(i)


qc3.measure_all()
qc3.draw('mpl')

then we transpile the circuit for the selected backend

In [ ]:
transpiled_qc3 = transpile(qc3, backend = hardware_backend)
transpiled_qc3.draw('mpl')

As you can see, transpilation greatly extend the depth of your circuit. You can use the options in the transpile function to reduce the depth of the transpiled circuit.

## Problem 4:  Quantum Fourier Transform

In [24]:
from qiskit.circuit.library import SGate, TGate
CS = SGate().control()
CT = TGate().control()

In [25]:
def qft(qc):
    # TODO implement the QFT for 3 qubits
    qc.h(0)
    qc.append(CS, [1,0])
    qc.append(CT, [2,0])
    qc.h(1)
    qc.append(CS, [2,1])
    qc.h(2)
    qc.swap(0,2)

    
def initialize(qc):
    # TODO initialize to states different from |000⟩ here
    qc.h(0)
    qc.x(1)
    qc.h(2)

In [26]:
# HINT:

# you can use

#from qiskit.circuit.library import SGate, TGate
#CS = SGate().control()
#CT = TGate().control()
#qc.append(CS, [control,target])
#qc.append(CT, [control,target])

# or a controlled phase gate specifying the angles for S and T in terms of π

# from math import pi
# qc.cp(angle, control, target)

In [ ]:
qc4 = QuantumCircuit(3)

initialize(qc4)
qft(qc4)
qc4.measure_all()

qc4.draw(output='mpl')

In [ ]:
results4 = execute(qc4, backend=simulator, shots=1000).result()
plot_histogram(results4.get_counts())

In [29]:
# TODO also run the qft cirquit with a noise model like in Problem 3 and plot the results

In [ ]:
results5 = execute(qc4, backend=noisy_qasm_simulator, shots=1000).result()
plot_histogram(results5.get_counts())